In [22]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json


In [23]:

path = 'E:\Technologie\Master\Webm\wem'

with open(f'{path}\\movies.json',encoding="utf8") as f:
    data = json.load(f)

with open(f'{path}\\nlp.json',encoding="utf8") as f:
    data_nlp = json.load(f)
 


In [24]:
df_nlp = pd.DataFrame(data_nlp).T
df_data = pd.DataFrame(data)

In [29]:
# set the uuid as index
df_data.set_index('uuid', inplace=True)

In [30]:
# join the two dataframes on the index and uuid
df = df_data.join(df_nlp, how="inner")

In [38]:
df.head()

,title,year,image,actors,plot,signature,ttr,fre,readability,tags
2327bfa4f30011edb451b8aeed79c0cc,Buffy the Vampire Slayer,1992,Buffy The Vampire Slayer Movie.jpg,"[Kristy Swanson, Luke Perry, Rutger Hauer, Pau...",Buffy Summers is a cheerleader at Hemery High ...,"[[0.4492585610019946, -0.10435049338123388, 0....",0.191702,98.62,2.0,"[action, journey, adventure]"
26c483fef30011edb451b8aeed79c0cc,Fearless,1993,Fearless ver1.jpg,"[ \n, Jeff Bridges, Isabella Rossellini, Rosie...",Max Klein survives an airline crash. The plane...,"[[0.36750598188921874, 0.028326033376961626, -...",0.154756,99.23,2.0,"[mystery, conspiracy, suspense]"
271c9396f30011edb451b8aeed79c0cc,Show Me Love,1998,Fucking Åmål original poster.jpg,"[Alexandra Dahlström, Rebecka Liljeberg, Eric...","Two girls, Agnes and Elin, attend school in th...","[[0.39478842674431, -0.017388587699599136, 0.0...",0.137574,99.23,3.0,"[business, action, journey]"
2b5896d0f30011edb451b8aeed79c0cc,Miss Congeniality,2000,Miss Congeniality Poster.jpg,"[ Sandra Bullock\n, Michael Caine, Benjamin Br...","In 1982, a young Gracie Hart steps into a play...","[[0.4135457351151562, 0.24593668651936273, -0....",0.203982,90.05,3.0,"[business, action, journey]"
2c134ed0f30011edb451b8aeed79c0cc,Mulholland Drive,2001,Mulholland.png,"[Naomi Watts, Justin Theroux, Laura Harring, L...",A dark-haired woman is the sole survivor of a ...,"[[0.31173353955770544, -0.013913081915504734, ...",0.170954,97.8,3.0,"[mystery, conspiracy, suspense]"
